# About
This notebook was developed by summer intern Pratyush (p.kumar-12@student.tudelft.nl) for his internship on "Development of python based pipeline for automated detection of deforestation using optical images from Sentinel 2"

The notebook when run, ultimately results in 3 different UI panels:
1. Daily mosaics display: takes input from user through a calendar for start date and end date, and displays the FCC and MBI for images within those dates
2. Monthly composite display: takes date range from user along with the number of months to be composited and the method of composition, displays the FCC and classified MBI on the map
3. NICFI PLANET display: takes date range from user (as calendar) and displays the NICFI basemap in FCC as well as NDVI and Reclassified NDVI


In [1]:
# IMPORTS
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
import datetime
# ee.Authenticate() #used for authenticating a GEE account, opens new tab in browser for signing into a google account
ee.Initialize() # initialize GEE

# Layer imports, sytling and all Global variables which can be changed from here

In [2]:
# BOUNDS AND STYLING 

aoi = ee.FeatureCollection("users/pratyush-s4g/reforester_bound")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
nicfi_raw = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/asia')

fnf = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/FNF") #ALOS PALSAR data if needed to be displayed
fnf_image = fnf.limit(1, 'system:time_start', False).first()


# define styling params
# Sentinel vis RGB and FCC
fccVis = { 'min': 0.0,    'max': 3000,      'bands': ['B8', 'B4', 'B3'] } # for styling sentinel 2 images as FCC
rgbVis = { 'min': 0.0,    'max': 3000,      'bands': ['B4', 'B3', 'B2'] } # for styling sentinel 2 images as RGB
# reclassed soil index vis
reclass_vis = {'min':-1, 'max':1, 'palette':['#f1a340','#7fbf7b'] }
# foresr/non-forest vis (ALOS PALSAR)
forestNonForestVis = {"min":1,  "max":3,    "palette":["006400","FEFF99","0000FF"] } #for styling FNF PALSAR2 map
# AOI styling
style =  { "color": "black",    "width": 1,    "fillColor": "00000111"} #for styling the ITCI boundary
styleParams = {  'fillColor': '00000111',  'color': '00909F',  'width': 3.0}
# NICFI
fccVisNICFI = {"bands":["N","R","G"], "min":64, "max":5454, "gamma":1.8}
ndviVisNICFI = { "bands":['NDVI'],'min':-0.55, 'max':0.8, 'palette': ['8bc4f9', 'c9995c', 'c7d270', '8add60', '097210' ]}
ndviReclassVisNICFI = {'min':-1, 'max':1, 'palette':['#7fbf7b','#f1a340'] }


# ####### GLOBAL VARIABLES
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50
CLOUD_FILTER = 100

# NICFI NDVI Thresholding
NICFI_NDVI_thresh_val = 0.66

# all thresholds are reclassified
THRESHOLDS_reclass_soil_indices = { "MBI":1.15, "NDMI":-0.1, "BSI": -0.29 } 

# start and end date definitions for main image collection, starts 2018 and goes on till next year
START_DATE = ee.Date('2018-01-01')
END_DATE = ee.Date('2023-12-01')

# Function definitions

In [3]:


# ####### CLOUD MASKING FUNCTIONS ############
def get_s2_sr_cld_col(aoi, start_date, end_date): 
    """
    summary: 

    aoi (ee.FeatureCollection): area of interest
    start_date(ee.Date): start date for filtering image collection
    end_date(ee.Date)  : end date for filtering image collection

    returns (ee.ImageCollection): cloud probability added to images
    """
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    """ 
    summary: takes image and adds cloud bands to it
    img (ee.Image) : image
    returns (ee.Image): ee image with added cloud bands
    """
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    """ 
    summary: takes image and adds shadow bands to it
    img (ee.Image) : image
    returns (ee.Image): ee image with added shadow bands
    """
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    """ 
    summary: takes image and masks shadow
    img (ee.Image) : image
    returns (ee.Image)
    """
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
    ## other option return img.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    """ 
    summary: takes image and masks cloud
    img (ee.Image) : image
    returns (ee.Image)
    """    
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# //#/////////// FUNCTIONS FOR INDICES ////////////
def add_indices(img):
    """ 
    summary: takes image and adds indices to ir (MBI: Modified bare soil index, BSI: Bare soil index, NDMI: Normalised Difference Moisture Index and BI)
    img (ee.Image) : image
    returns (ee.Image) with added indices as bands
    """
    swir1= img.select('B11')
    swir2= img.select('B12')
    nir_b8a  = img.select('B8A')
    nir_b8 = img.select('B8')
    blue = img.select('B2')
    red = img.select('B4')


    # //////////// MBI
    mbi_numerator = swir1.subtract(swir2.subtract(nir_b8a))
    mbi_denominator = swir1.add(swir2.add(nir_b8a))
    mbi_image = ( mbi_numerator.divide(mbi_denominator).add(0.5)).reproject(**{'crs': img.select([0]).projection(), 'scale': 20}).rename('MBI')
    # add mbi band
    img = img.addBands(mbi_image)

    # /////////// BI   
    bi = (swir1.subtract(swir2)).divide( (swir1.add(swir2)) )
    # add bi band
    img = img.addBands(bi.reproject(**{'crs': img.select([0]).projection(), 'scale': 20}).rename('BI'))

    # /////////// BSI
    bsi = ( (swir2.add(red)).subtract(nir_b8.add(blue)) ).divide( (swir2.add(red)).add(nir_b8.add(blue)) )
    # add bi band
    img = img.addBands(bsi.reproject(**{'crs': img.select([0]).projection(), 'scale': 20}).rename('BSI'))

    # ////////////// NDWI
    ndmi = img.normalizedDifference(['B8','B11'])
    # add bi band
    img = img.addBands(ndmi.reproject(**{'crs': img.select([0]).projection(), 'scale': 20}).rename('NDMI'))

    return img

# /////////////////////////
def water_remover(img):
    """
    summary: takes image and masks those areas which have probability of water as per the JRC Global Surface Water layer on GEE
    img (ee.Image)
    returns (ee.Image): image with water areas masked
    """
    mask = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').unmask(-99999)
    noWaterImg = img.updateMask(mask.eq(-99999))
    return noWaterImg

# ///////////////////////////////////// NICFI NDVI
def nicfiNDVI_maker(img):
    """
    summary: takes NICFI PLANET high res image and computes NDVI, reclasses NDVI by threshold value, adds these as bands
    img (ee.Image)
    returns (ee.Image)
    """
    ndvi = ee.Image(img).normalizedDifference(['N','R'])
    img = img.addBands(ndvi.reproject(**{'crs': img.select([0]).projection(), 'scale': img.select([0]).projection().nominalScale()}).rename('NDVI'))

    ndvi = img.select('NDVI')
    reclassedImg = ee.Image(-9999).where( ndvi.gte(NICFI_NDVI_thresh_val) , -1 ).where( ndvi.lt(NICFI_NDVI_thresh_val) , 1)
    mask = reclassedImg.neq(0)
    reclassedImg=reclassedImg.updateMask(mask).rename("NDVI_reclassed")
    img = img.addBands(reclassedImg)
    return img


#//////////////////////// /////////// define thresholds
# new approach, image has all index bands, and dict taken as param for thresholds
def reclass_entire_image(image):
    """
    summary: takes image and reclassifies the BSI, MBI and NDMI bands as per the globally defined thresholds
    image (ee.Image)
    return (ee.Image): with reclassifications added as new bands, each with Index name and suffix of '_reclassed'
    """
    for thresh_type, thresh_val in THRESHOLDS_reclass_soil_indices.items():
        img_raw = image.select(thresh_type)

        if thresh_type != 'BSI':    
            reclassedImg = ee.Image(0).where( img_raw.lt(thresh_val) , -1 ).where( img_raw.gte(thresh_val) , 1)
            mask = reclassedImg.neq(0)
            reclassedImg=reclassedImg.updateMask(mask).rename(thresh_type+'_reclassed')

        elif thresh_type == 'BSI':    
            reclassedImg = ee.Image(0).where( img_raw.gt(thresh_val) , -1 ).where( img_raw.lte(thresh_val) , 1)
            mask = reclassedImg.neq(0)
            reclassedImg=reclassedImg.updateMask(mask).rename(thresh_type+'_reclassed')

        # add reclassified image as a band to original image source
        image = image.addBands(reclassedImg)
    return image


# /////////////////////////////////   # change detection
# compare this image with a previous one in the dataframe and check if it was already bare soil
# make function which takes 2 images and then checks for deforestation
def change_detector(img1, img2):
    """
    summary: takes 2 images as earth engine images, and checks if something changed from not-bare soil to bare soil
    img1 (ee.Image): image of date < img2
    img2 (ee.Image): image of date > img1
    return: returns a new image with only those pixels which changed from non-bare soil to bare soil
    """
    # get mask
    mask1 = img1.neq(0)
    mask2 = img2.neq(0)
   
    # convert the images to boolean , True if bare soil
    img1_bool = ee.Image(0).where(img1.eq(-1), 1)
    img2_bool = ee.Image(0).where(img2.eq(-1), 1)

    new_image = ee.Image(0).where( (img2_bool.eq(1).And( img1_bool.eq(0)) ) , 1 )
    new_image = new_image.updateMask(mask1).updateMask(mask2)
    # print(mask1)
    return new_image


# mosaick by day function
def mosaicByDate(imcol):
    """
    summary: takes image collection and mosaics images in it by day, so single image per aoi per day
    imcol (ee.ImageCollection) : image collection filtered by date by user
    return (ee.ImageCollection) : mosaiced by day
    """
    def mosaicking_func_helper(d):
        d = ee.Date(d)
        im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
        return im.set(
            "system:time_start", d.millis(), 
            "system:id", d.format("YYYY-MM-dd"))

    imlist = imcol.toList(imcol.size())
    unique_dates = imlist.map( lambda im: ee.Image(im).date().format("YYYY-MM-dd")  ).distinct()
    mosaic_imlist = unique_dates.map(mosaicking_func_helper)
    return ee.ImageCollection(mosaic_imlist)

  # mosaick by day function

def compositeByMonth(imcol, no_of_months=1, mode='median'):
  """
    summary: for monthly composite, the user inputs the date duration and the number of months to composite as well as the mode of composition
             code then takes the mega imageCollection and filters by date
             after this the code then keeps only list of months which need to be kept for the composition function to run on every nth month
    imcol (ee.ImageCollection): image collection of daily mosaics to be composited together by month
    no_of_months (int)        : number of months, whose images need to be composited together
    mode (str)                : method of making composite
  returns: new image collection of images composited together
  """
  def mosaicking_func_helper_month(d):
    d = ee.Date(d)
    # im = imcol.filterDate( d.advance(no_of_months*(-1), "month") , d.advance(1,'month').advance(-1,'day') ).median()
    if mode=='median':
        im = imcol.filterDate( d.advance(no_of_months*(-1), "month") , d.advance(1,'month').advance(-1,'day') ).median()
    elif mode=='mean':
        im = imcol.filterDate( d.advance(no_of_months*(-1), "month") , d.advance(1,'month').advance(-1,'day') ).mean()
    elif mode=='mosaic':
        im = imcol.filterDate( d.advance(no_of_months*(-1), "month") , d.advance(1,'month').advance(-1,'day') ).mosaic()
    return im.set("system:time_start", d.millis(), "system:id", d.format("YYYY-MM"))
      
  imlist = imcol.toList(imcol.size())
  unique_dates = imlist.map( lambda im: ee.Image(im).date().format("YYYY-MM")).distinct().reverse() #reverses order of the list of unique months
  datesIndx = ee.List.sequence(0, ee.Number.subtract(unique_dates.size() ,ee.Number(1)), no_of_months )
  finDates = datesIndx.map(lambda x: unique_dates.get(x))
  no_of_months-=1
  #   make list of dates 
  if int(unique_dates.size().getInfo()) <= no_of_months+1:
    no_of_months = 1 #return monthly mosaic if date invalid
    mosaic_imlist = finDates.map(mosaicking_func_helper_month)
  else:
    mosaic_imlist = finDates.map(mosaicking_func_helper_month)
  
  return ee.ImageCollection(mosaic_imlist)

# ///////////////////// function for displaying iteratively an image collection
def display_func(imgCol, aoi, method= 'monthly'):
    """
    summary: displays the image collections' images and MBI on a geemap.Map instance
    
    imgcol (ee.ImageCollection) : image collection whose images are to be displayed
    aoi (ee.FeatureCollection)  : Area of interest
    method (str)                : defines if the image collection is of type (daily mosaic, monthly composite or NICFI PLANET high resolution images) 
                                if the image collection is of type NICFI then the FCC and the NDVI and reclassified NDVI are displayed
    
    return (geemap.Map()): a geemap.Map instance based on ipyLeaflet with added FCC images and MBI reclassifications showing bare soil/non bare soil
    """
    if method == 'monthly':
        imgColDisp = geemap.Map()
        imgColDisp.centerObject(aoi,9.5)
        imgColAsList = imgCol.toList(imgCol.size())
        for i in range(int(imgColAsList.size().getInfo())):
            img= ee.Image(imgColAsList.get(i))
            img_dt = img.date().format('YYYY-MM').getInfo()
            imgColDisp.addLayer( img.clip(aoi) , fccVis, img_dt)
            imgColDisp.addLayer( img.clip(aoi).select(['MBI_reclassed']) , reclass_vis, img_dt+'_MBI_reclassified')
        imgColDisp.addLayer(aoi.style(**style), {}, "aoi")
        # imgCol.map(lambda x: imgColDisp.addLayer(x, fccVis)) # doesnt work this way
        return imgColDisp

    elif method == 'daily': # not monthly mosaic images , so consider that date range will have day as well
        imgColDisp = geemap.Map()
        imgColDisp.centerObject(aoi,9.5)
        imgColAsList = imgCol.toList(imgCol.size())
        for i in range(int(imgColAsList.size().getInfo())):
            img= ee.Image(imgColAsList.get(i))
            img_dt = img.date().format('YYYY-MM-dd').getInfo()
            imgColDisp.addLayer( img.clip(aoi) , fccVis, img_dt)
            imgColDisp.addLayer( img.clip(aoi).select(['MBI_reclassed']) , reclass_vis, img_dt+'_MBI_reclassified')
        imgColDisp.addLayer(aoi.style(**style), {}, "aoi")
        return imgColDisp

    elif method == 'nicfi': # not monthly mosaic images , so consider that date range will have day as well
        imgColDisp = geemap.Map()
        imgColDisp.centerObject(aoi,9.5)
        imgColAsList = imgCol.toList(imgCol.size())
        for i in range(int(imgColAsList.size().getInfo())):
            img= ee.Image(imgColAsList.get(i))
            img_dt = img.date().format('YYYY-MM-dd').getInfo()
            imgColDisp.addLayer( img.clip(aoi) , fccVisNICFI,  img_dt )
            imgColDisp.addLayer( img.clip(aoi) , ndviVisNICFI, img_dt + '_NDVI')
            imgColDisp.addLayer( img.select('NDVI_reclassed').clip(aoi) , ndviReclassVisNICFI, img_dt + '_NDVI_reclass')
        imgColDisp.addLayer(aoi.style(**style), {}, "aoi")
        return imgColDisp

# ////////////////
def compute_pol_area(pol):
    """
    summary: computes area of the polygon 
    pol (ee.FeatureCollection): polygon, (MBI or NDVI reduced to vector)
    return (ee.FeatureCollection)
    """
    proj = 'EPSG:32750'
    area_ha = pol.area( **{'maxError':1, 'proj':ee.Projection(proj)} ).divide(10000)
    return pol.set('area_ha',area_ha)

def exporting_function(imgCol, aoi, planet=False):
    """
    summary: takes image collecyion and computes, the vector for soil/non soil or NDVI and exports the raster and vector
    imgcol (ee.ImageCollection): image collection whose images are to be exported, can be monthly composites, 
                                daily mosaics or NICFI maps with NDVI
    aoi (ee.FeatureCollection) : are of interest
    planet (Boolean)           : if true, it indicates that the image collection is from NICFI, otherwise is treated like 
                                a Sentinel 2 image with added reclassified soil indices

    return (str): says that exporting was run
    """
    imgColAsList = imgCol.toList(imgCol.size())
    for i in range(int(imgColAsList.size().getInfo())):
        img= ee.Image(imgColAsList.get(i)).clip(aoi)
        if planet == False: #means S2 data
            img_dt = img.date().format('YYYY-MM-dd').getInfo()
            # get vector
            soil = ee.Image(0).where(img.select('MBI_reclassed').eq(-1), 1)
            soil = soil.mask(soil.neq(0))
            # vector = soil.reduceToVectors(**{'reducer':'mean', 'scale':20, 'eightConnected':True , 'maxPixels':79199549900}
            vector = soil.reduceToVectors(**{
                                    'geometry': aoi.geometry(),
                                    'crs': soil.projection(),
                                    'scale': 20,
                                    'geometryType': 'polygon',
                                    'eightConnected': True,
                                    'labelProperty': 'zone',
                                    'maxPixels':79199549900
                                    }).map(compute_pol_area).filterMetadata('area_ha','greater_than',0.01)
            # Export the image, specifying scale and region.
            task_fcc = ee.batch.Export.image.toDrive(**{
                                                        'image': img.select(['B8','B4','B3','B2']),
                                                        'description': "FCC_RGB_"+img_dt,
                                                        'folder':'Exports',
                                                        'scale': 10,
                                                        'region': aoi.geometry(),
                                                        'crs':img.select(["B4"]).projection()
                                                    })
            task_soil_vect = ee.batch.Export.table.toDrive(**{
                                                            'collection': vector,
                                                            'description': "MBI_" + img_dt,
                                                            'folder':'Exports',
                                                            'fileFormat': 'GeoJSON'
                                                            })
            task_fcc.start()
            task_soil_vect.start()
        else: #planet data
            img_dt = img.date().format('YYYY-MM').getInfo()
            
            # get vector
            soil = img.select('NDVI_reclassed').eq(1)
            soil = soil.mask(soil.neq(0))
            # vector = soil.reduceToVectors(**{'reducer':'mean', 'scale':img.select(['NDVI_reclassed']).projection().nominalScale(), 'eightConnected':True , 'maxPixels':79199549900})
            vector = soil.reduceToVectors(**{
                                    'geometry': aoi.geometry(),
                                    'crs': img.select(['N']).projection(),
                                    'scale': img.select(['N']).projection().nominalScale(),
                                    'geometryType': 'polygon',
                                    'eightConnected': True,
                                    'labelProperty': 'zone',
                                    'maxPixels':79199549900
                                    }).map(compute_pol_area).filterMetadata('area_ha','greater_than',0.01)
            # Export the image, specifying scale and region.
            task_fcc = ee.batch.Export.image.toDrive(**{
                                                        'image': img.select(['N','R','G','B']),
                                                        'description': "NICFI_"+img_dt,
                                                        'folder':'Exports',
                                                        'scale': img.select([0]).projection().nominalScale(),
                                                        'region': aoi.geometry(),
                                                        'crs':img.select(['N']).projection(),
                                                        'maxPixels': 200000000
                                                    })
            task_soil_vect = ee.batch.Export.table.toDrive(**{
                                                            'collection': vector,
                                                            'description': "NDVI_" + img_dt,
                                                            'folder':'Exports',
                                                            'fileFormat': 'GeoJSON'
                                                            })
            task_fcc.start()
            task_soil_vect.start()


    return "....Export has begun...."

# Image collection formation
Initially a pandas DataFrame baseed approach was used, which can be found in initial commits, it worked fine, however was super slow owing to the usage of .getInfo() on every image in the image collection. Moreover, for that method to work, a scheduled task would have to be run to keep the main datafrsame updated with all images.

taking in input from [stackoverflow](https://gis.stackexchange.com/questions/280156/mosaicking-a-image-collection-by-date-day-in-google-earth-engine)

Qn: Why were we using the dataframe approach and what advantages does it have over just using an imageCollaction and filtering it out by day

Ans:
* maybe was better to use df as it is easy to groupby date and mosaic 
* or just extracting images from a single storage location is easy
* easy to share the images inside the team since the processed images can alltogether be stored in as less as 50KB

image collection for ITCI boundary with S2 is available from 2018 december, best to make collection from then
could also use it as a limitor if user inputs the date

## Thresholding
All image collections are defined below, namely:
1. nicfi_with_NDVI: PLANET basemap (NRGB) mapped with water removal and NDVI, reclassified NDVI band added
2. s2_cloud_masked: S2 level 2A image collection with cloud probabilities and shadows added
3. s2_cloud_masked: above image collection without clouds and shadows
4. mosaicked_img_col: daily mosaicked images with water removed and soil indices added
5. mosaicked_img_col_thresholded: above with reclassified soil indices added

In [4]:
aoi_new = ee.Geometry.LineString( [[116.35173971508155, -0.9258534575298593], [116.43001730297217, -0.7034029388578974]])
# NICFI PLANET
nicfi_with_NDVI = nicfi_raw.map( nicfiNDVI_maker).map(water_remover)
# Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
s2_cloud_masked = get_s2_sr_cld_col(aoi_new, START_DATE, END_DATE)
# adding cloud masking
cloud_masked_collection = s2_cloud_masked.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)
# mosaicing by date and adding indices, removing water
mosaicked_img_col = mosaicByDate(cloud_masked_collection).map(add_indices).map(water_remover)
# applying the thresholds as defined in the function definition
mosaicked_img_col_thresholded = mosaicked_img_col.map(reclass_entire_image)

# UI with jupyter widgets
The below cell defines different widgets to be used for display of images using UI, to be used both for daily mosaics and monthly composites
It also goes ahead and sets up the GridSpecs for displaying these UI widgets in a grid fashion

In [5]:
month_num = list(zip( "January February March April May June July August September October November December".split(), [int(x) for x in "1 2 3 4 5 6 7 8 9 10 11 12".split()] ))
yearNumList = list(range(2018,2022))
datesNumList = list(range(1,32))

numOfMonnthsCompositeInput = widgets.IntText(value=2,description='Select Number of months for the composite to be made',disabled=False,orientation= 'center')
compositeType = widgets.RadioButtons(options=['median', 'mean', 'mosaic'],value='median', description='Method:',disabled=False)

dailyMosDateStart = widgets.DatePicker(description='Start Date', disabled=False, value = datetime.date(2020,12,15) )
dailyMosDateEnd   = widgets.DatePicker(description='End Date'  , disabled=False, value = datetime.date(2021,1,15) )

monCompDateStart =  widgets.DatePicker(description='Start Date', disabled=False, value = datetime.date(2021,1,1) )
monCompDateEnd   =  widgets.DatePicker(description='End Date'  , disabled=False, value = datetime.date(2021,10,31) )

nicfiDateStart =    widgets.DatePicker(description='Start Date', disabled=False, value = datetime.date(2019,8,1) )
nicfiDateEnd   =    widgets.DatePicker(description='End Date'  , disabled=False, value = datetime.date(2020,6,30) )
testButt = widgets.Box()
grid_day_mos = widgets.GridspecLayout(2,2)
grid_day_mos[0,0] = dailyMosDateStart
grid_day_mos[1,0] = dailyMosDateEnd
# [:,1] = 

grid_mon_comp = widgets.GridspecLayout(3,4)
grid_mon_comp[0,:2] = monCompDateStart
grid_mon_comp[1,:2] = monCompDateEnd
grid_mon_comp[2:,:2 ] = numOfMonnthsCompositeInput
grid_mon_comp[:,2: ] = compositeType

grid_nicfi = widgets.GridspecLayout(2,2)
grid_nicfi[0,:] = nicfiDateStart
grid_nicfi[1,:] = nicfiDateEnd


# Daily Mosaic with user interaction

In [6]:
grid_day_mos

GridspecLayout(children=(DatePicker(value=datetime.date(2020, 12, 15), description='Start Date', layout=Layout…

In [7]:
dt_begin_user, dt_end_user = ee.Date(dailyMosDateStart.value.strftime('%Y-%m-%d')), ee.Date(dailyMosDateEnd.value.strftime('%Y-%m-%d'))
coln = mosaicked_img_col_thresholded.filterDate(dt_begin_user, dt_end_user)
display_func(coln, aoi, 'daily')

Map(center=[-0.75764352288308, 116.5124147639097], controls=(WidgetControl(options=['position', 'transparent_b…

In [8]:
# //////////// FOR EXPORTING, takes user input using an input tab
y_n= (input("Are you sure you want to export the Daily Mosaics Image collection and MBI to your Drive? y/[n]: ")).lower()
if y_n == 'y':
    # yes, do export
    exporting_function(coln, aoi, planet=False)
else:
    print("You did not say Yes to exporting, aborting export...")

You did not say Yes to exporting, aborting export...


# Monthly composition with user interaction

In [9]:
grid_mon_comp

GridspecLayout(children=(DatePicker(value=datetime.date(2021, 1, 1), description='Start Date', layout=Layout(g…

In [10]:
stDt, edDt = ee.Date(monCompDateStart.value.strftime('%Y-%m-%d')), ee.Date(monCompDateEnd.value.strftime('%Y-%m-%d'))
user_month_input = numOfMonnthsCompositeInput.value
method = compositeType.value
# display_func(mosaicked_img_col_thresholded.filterDate(dt_begin_user, dt_end_user), aoi, False) 
monMos = compositeByMonth(mosaicked_img_col.filterDate(stDt, edDt) ,user_month_input, method)
monmos_thresholded = monMos.map(reclass_entire_image)
display_func(monmos_thresholded, aoi, 'monthly')

Map(center=[-0.75764352288308, 116.5124147639097], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
# ///////////// UNCOMMENT BELOW LINE FOR ENABLING EXPORT
y_n= (input("Are you sure you want to export the Daily Mosaics Image collection and MBI to your Drive? y/[n]: ")).lower()
if y_n == 'y':
    # yes, do export
    exporting_function(monmos_thresholded, aoi, False)
else:
    print("You did not say Yes to exporting, aborting export...")

You did not say Yes to exporting, aborting export...


# NICFI FCC and NDVI

In [12]:
grid_nicfi

GridspecLayout(children=(DatePicker(value=datetime.date(2019, 8, 1), description='Start Date', layout=Layout(g…

In [13]:
nicfi_dt_begin_user, nicfi_dt_end_user = ee.Date(nicfiDateStart.value.strftime('%Y-%m-%d')), ee.Date(nicfiDateEnd.value.strftime('%Y-%m-%d'))
coln_nicfi = nicfi_with_NDVI.filterDate(nicfi_dt_begin_user, nicfi_dt_end_user) 
display_func( coln_nicfi, aoi, 'nicfi')

Map(center=[-0.75764352288308, 116.5124147639097], controls=(WidgetControl(options=['position', 'transparent_b…

In [14]:
y_n= (input("Are you sure you want to export the Daily Mosaics Image collection and MBI to your Drive? y/[n]: ")).lower()
if y_n == 'y':
    # yes, do export
    exporting_function(coln_nicfi, aoi, planet=True)
else:
    print("You did not say Yes to exporting, aborting export...")

You did not say Yes to exporting, aborting export...


# Tile URL in case needed (Example)
From this [link](https://gis.stackexchange.com/questions/355014/ipyleaflet-with-gee-in-jupyterlab-doesnt-show-layer), taking the answer, we get the Tile URL of the image that we wish to display

This tile URL can be further used in a Mapbox or DeckGL based UI

In [15]:
# nicfi tile url
testIDImage = nicfi_with_NDVI.filterDate(ee.Date('2019-06-01'), ee.Date('2019-10-01')).first()
testIDImage.getMapId({'bands': ['N', 'R', 'G'], 'min': 64, 'max': 5454, 'gamma': 1.8})['tile_fetcher'].url_format

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/49e67bee21a205aad336082ecacc3b99-2c2e1a7651541042f2e4fc3fdd9ffdef/tiles/{z}/{x}/{y}'